In [19]:
import os

In [20]:
import torch.autograd
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import save_image

In [21]:
# Create folder if not exist
if not os.path.exists('./img'):
  os.mkdir('./img')

In [22]:
# deal with images
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [23]:
# Download MNist dataset
mnist = datasets.MNIST(
    root='./data/',
    train = True,
    transform= img_transform,
    download=True
)

In [24]:
print(mnist.train_data.size())                 # (60000, 28, 28)
print(mnist.train_labels.size())

torch.Size([60000, 28, 28])
torch.Size([60000])


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:52: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [25]:
# Hyperparameters
batch_size = 128

num_epoch = 100

z_dimension = 100

lr = 0.0003

In [26]:
dataloader = torch.utils.data.DataLoader(
    dataset = mnist,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 2
)

**Discriminator**

In [27]:
class discriminator(nn.Module):
  def __init__(self):
    super(discriminator,self).__init__()
    self.main = nn.Sequential(
        nn.Linear(28*28, 256),  # linear
        nn.LeakyReLU(negative_slope=0.2),  # relu
        nn.Linear(256, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 1),
        nn.Sigmoid()

    )
  def forward(self, input):
    return self.main(input)

**Generator**

In [28]:
class generator(nn.Module):
  def __init__(self):
    super(generator,self).__init__()
    self.main = nn.Sequential(
        nn.Linear(100, 256),  # Linear
        nn.ReLU(inplace=True),
        nn.Linear(256,256), # Linear
        nn.ReLU(inplace=True),
        nn.Linear(256,28*28),
        nn.Tanh()
    )
  def forward(self, input):
    return self.main(input)

**Model**

In [29]:
D = discriminator()
G = generator()

In [30]:
if torch.cuda.is_available():
  D = D.cuda()
  G = G.cuda()

**Loss and Optimizer**

In [31]:
loss_func = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr)

**Training**

In [32]:
def to_img(input):
  out = 0.5*(input+1)
  out = out.clamp(0,1)  # make out within [min,max]
  out = out.view(-1,1,28,28)
  return out

In [33]:
for epoch in range(num_epoch):
  for step, (image,_) in enumerate(dataloader):
    num_img = image.size(0)

    img = image.view(num_img, -1)  # n,d
    real_img = Variable(img).cuda()

    real_label = Variable(torch.ones(num_img)).cuda() # real image has label of one
    fake_label = Variable(torch.zeros(num_img)).cuda()  # generated image has label of zero

    # ############# Train discriminator #################
    # Two parts: 1. real is real 2. fake is fake
    # Calculate loss
    real_out = D(real_img)
    real_out = real_out.squeeze() # (128,1) -> (128,)
    d_loss_real = loss_func(real_out,real_label)  # loss for real image
    real_scores = real_out
    # loss for fake images
    z = Variable(torch.randn(num_img,z_dimension)).cuda() # random noise
    fake_img = G(z).detach()  # random generte pictures from the noise  # use detach to prevent gradient to be delivered in G
    fake_out = D(fake_img)  # discriminator to judge if fake is fake
    fake_out = fake_out.squeeze() # (128,1) -> (128,)
    d_loss_fake = loss_func(fake_out,fake_label)  # loss for fake image
    fake_scores = fake_out
    # Total Loss
    d_loss = d_loss_real + d_loss_fake

    d_optimizer.zero_grad()
    d_loss.backward()
    d_optimizer.step()

    # ###################### Train Generator ########################
    # Fix discriminator. train gernerator to generate image let discriminator judge to be real image
    z = Variable(torch.randn(num_img, z_dimension)).cuda()  # random noise
    fake_img = G(z) # fake image
    output = D(fake_img)  # discriminator to judge the image
    output = output.squeeze()
    g_loss = loss_func(output,real_label) # loss between real and fake

    g_optimizer.zero_grad()
    g_loss.backward()
    g_optimizer.step()

    # print loss
    if (step + 1) % 100 == 0:
      print('Epoch[{}/{}],d_loss:{:.6f},g_loss:{:.6f} '
            'D real: {:.6f},D fake: {:.6f}'.format(
          epoch, num_epoch, d_loss.data.item(), g_loss.data.item(),
          real_scores.data.mean(), fake_scores.data.mean()  # 打印的是真实图片的损失均值
      ))
    if epoch == 0:
        real_images = to_img(real_img.cpu().data)
        save_image(real_images, './img/real_images.png')
fake_images = to_img(fake_img.cpu().data)
save_image(fake_images, './img/fake_images-{}.png'.format(epoch + 1))


Epoch[0/100],d_loss:0.143944,g_loss:3.205808 D real: 0.971831,D fake: 0.107208
Epoch[0/100],d_loss:0.016940,g_loss:5.440063 D real: 0.994402,D fake: 0.011182
Epoch[0/100],d_loss:0.148669,g_loss:6.355597 D real: 0.959354,D fake: 0.086063
Epoch[0/100],d_loss:0.002924,g_loss:8.887615 D real: 0.998813,D fake: 0.001707
Epoch[1/100],d_loss:0.385592,g_loss:4.025088 D real: 0.914168,D fake: 0.206580
Epoch[1/100],d_loss:0.163897,g_loss:7.639924 D real: 0.974069,D fake: 0.087011
Epoch[1/100],d_loss:0.103485,g_loss:6.846142 D real: 0.944112,D fake: 0.009085
Epoch[1/100],d_loss:0.240253,g_loss:5.603417 D real: 0.983204,D fake: 0.185811
Epoch[2/100],d_loss:0.217489,g_loss:5.591542 D real: 0.946112,D fake: 0.100044
Epoch[2/100],d_loss:0.096481,g_loss:7.108648 D real: 0.956379,D fake: 0.025202
Epoch[2/100],d_loss:0.300175,g_loss:6.891470 D real: 0.910006,D fake: 0.097761
Epoch[2/100],d_loss:0.286694,g_loss:7.144184 D real: 0.906629,D fake: 0.043070
Epoch[3/100],d_loss:1.025300,g_loss:2.505839 D real:

**Save Model**

In [34]:
torch.save(G.state_dict(), './generator.pth')
torch.save(D.state_dict, './discriminator.pth')